<a href="https://colab.research.google.com/github/ParitoshP702/Bilevel-Optimization/blob/main/Grid_Search_without_lp(CIFAR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense
import random
from tqdm import tqdm
import numpy as np
import pandas as pd

In [ ]:
(X_train,Y_train),(X_test,Y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
train_len = 3500
eval_len = 1000
test_len = 1000

In [ ]:
x_train = X_train[:train_len,:,:,:]
x_eval = X_train[train_len:train_len+eval_len,:,:,:]
x_test = X_test[:test_len,:,:,:]
y_train = Y_train[:train_len]
y_eval = Y_train[train_len:train_len+eval_len]
y_test = Y_test[:test_len]

In [ ]:
x_train = x_train.reshape(x_train.shape[0],-1)
x_eval = x_eval.reshape(x_eval.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)

In [ ]:
y_training = np.zeros(shape = (len(y_train),10), dtype = float)#one hot encoding the training labels
for i in range(len(y_train)):
  for j in range(10):
    if j  == y_train[i]:
      y_training[i][j] = 1.0
    else:
      y_training[i][j] = 0.0

In [ ]:
y_val_array = np.zeros(shape = (len(y_eval),10),dtype  =float)#one hot encoding the validation labels
for i in range(len(y_eval)):
  for j in range(10):
    if j  == y_eval[i]:
      y_val_array[i][j] = 1.0
    else:
      y_val_array[i][j] = 0.0

In [ ]:
y_testing = np.zeros(shape = (len(y_test),10),dtype  =float)#one hot encoding the validation labels
for i in range(len(y_test)):
  for j in range(10):
    if j  == y_test[i]:
      y_testing[i][j] = 1.0
    else:
      y_testing[i][j] = 0.0

In [ ]:
def generate_population():
  neurons_per_layer = [5,10,15]
  number_of_layers = [1,2,3]
  activation_function = ["relu","sigmoid","tanh"]
  regularization_param = [1e-10,1e-9,1e-8]
  param_dict = []
  for i in range(len(neurons_per_layer)):
    for j in range(len(number_of_layers)):
      for k in range(len(activation_function)):
        for l in range(len(regularization_param)):
          dic = [neurons_per_layer[i],number_of_layers[j],activation_function[k],regularization_param[l]]
          param_dict.append(dic)
  return param_dict


In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
def train_model(parameters,initialWeights=None):
    neurons_per_layer = parameters[0]
    no_of_layers = parameters[1]
    activation_function = parameters[2]

    #Following is not used here
    # optimization_method = parameters[3]
    regularization_param = parameters[3]

    model = Sequential()
    model.add(Dense(units=2, input_dim=3072))

    for _ in range(no_of_layers):
        model.add(Dense(units=neurons_per_layer, activation=activation_function,kernel_regularizer = tf.keras.regularizers.L2(regularization_param)))

    model.add(Dense(units = 10,  activation = 'softmax',kernel_regularizer = tf.keras.regularizers.L2(regularization_param)))




    return(model)

def evaluate_model(individual,initialWeights=None):
    model = train_model(individual,initialWeights)

    #The last element in the individual should always be the optimizer
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="Adam", metrics=['accuracy'])
    model.fit(x_train, y_training, batch_size = 64, epochs = 10)

    # print("Training Accuracy:", model.evaluate(x_train, y_train, verbose = 0)[1])

    #Evaluate on evaluation data
    # loss_score, accuracy_score = model.evaluate(x_eval, y_val_array, verbose = 0)
    y_pred = model(x_eval,training = False)
    loss_score = loss_object(y_pred,y_val_array)
    return loss_score

In [ ]:
initial_population

[[5, 1, 'relu', 1e-10],
 [5, 1, 'relu', 1e-09],
 [5, 1, 'relu', 1e-08],
 [5, 1, 'sigmoid', 1e-10],
 [5, 1, 'sigmoid', 1e-09],
 [5, 1, 'sigmoid', 1e-08],
 [5, 1, 'tanh', 1e-10],
 [5, 1, 'tanh', 1e-09],
 [5, 1, 'tanh', 1e-08],
 [5, 2, 'relu', 1e-10],
 [5, 2, 'relu', 1e-09],
 [5, 2, 'relu', 1e-08],
 [5, 2, 'sigmoid', 1e-10],
 [5, 2, 'sigmoid', 1e-09],
 [5, 2, 'sigmoid', 1e-08],
 [5, 2, 'tanh', 1e-10],
 [5, 2, 'tanh', 1e-09],
 [5, 2, 'tanh', 1e-08],
 [5, 3, 'relu', 1e-10],
 [5, 3, 'relu', 1e-09],
 [5, 3, 'relu', 1e-08],
 [5, 3, 'sigmoid', 1e-10],
 [5, 3, 'sigmoid', 1e-09],
 [5, 3, 'sigmoid', 1e-08],
 [5, 3, 'tanh', 1e-10],
 [5, 3, 'tanh', 1e-09],
 [5, 3, 'tanh', 1e-08],
 [10, 1, 'relu', 1e-10],
 [10, 1, 'relu', 1e-09],
 [10, 1, 'relu', 1e-08],
 [10, 1, 'sigmoid', 1e-10],
 [10, 1, 'sigmoid', 1e-09],
 [10, 1, 'sigmoid', 1e-08],
 [10, 1, 'tanh', 1e-10],
 [10, 1, 'tanh', 1e-09],
 [10, 1, 'tanh', 1e-08],
 [10, 2, 'relu', 1e-10],
 [10, 2, 'relu', 1e-09],
 [10, 2, 'relu', 1e-08],
 [10, 2, 'sigmoi

In [ ]:
initial_population = generate_population()
losses = []
# models = []
# losses = [evaluate_model(individual) for individual in initial_population]
for individual in initial_population:
  loss = evaluate_model(individual)
  losses.append(loss)

Epoch 1/10
55/55 [==============================] - 1s 4ms/step - loss: 47.4826 - accuracy: 0.1131
Epoch 2/10
55/55 [==============================] - 0s 4ms/step - loss: 11.6032 - accuracy: 0.1174
Epoch 3/10
55/55 [==============================] - 0s 4ms/step - loss: 3.4532 - accuracy: 0.1086
Epoch 4/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3412 - accuracy: 0.1071
Epoch 5/10
55/55 [==============================] - 0s 3ms/step - loss: 2.3278 - accuracy: 0.1074
Epoch 6/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3159 - accuracy: 0.1077
Epoch 7/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3081 - accuracy: 0.1074
Epoch 8/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3055 - accuracy: 0.1074
Epoch 9/10
55/55 [==============================] - 0s 3ms/step - loss: 2.3046 - accuracy: 0.1074
Epoch 10/10
55/55 [==============================] - 0s 3ms/step - loss: 2.3027 - accuracy: 0.1077
Epoch 1/10
55/55 

In [ ]:
np.array(losses).min()

13.845140279841683

In [ ]:
best_param = initial_population[np.array(losses).argmin()]

In [ ]:
reg = best_param[3]
number_of_layers=  best_param[0]
neuron_per_layer = best_param[1]
activation_function = best_param[2]


In [ ]:
model = Sequential()
model.add(Dense(units = 2,input_dim = 3072 ))
for i in range(number_of_layers):
  model.add(Dense(units = neuron_per_layer, activation = activation_function, kernel_regularizer = tf.keras.regularizers.L2(reg)))
model.add(Dense(units = 10,activation = "softmax", kernel_regularizer = tf.keras.regularizers.L2(reg) ))

In [ ]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer = "Adam", metrics = ["accuracy"])
model.fit(x_train,y_training, epochs = 10,batch_size = 64)
y_pred = model(x_eval,training = False)
loss_object(y_pred,y_val_array).numpy()##loss on validation
_,accuracy = model.evaluate(x_eval,y_val_array,verbose = 0)

Epoch 1/10
55/55 [==============================] - 1s 4ms/step - loss: 2.3027 - accuracy: 0.0994
Epoch 2/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3025 - accuracy: 0.1080
Epoch 3/10
55/55 [==============================] - 0s 5ms/step - loss: 2.3024 - accuracy: 0.1080
Epoch 4/10
55/55 [==============================] - 0s 5ms/step - loss: 2.3023 - accuracy: 0.1080
Epoch 5/10
55/55 [==============================] - 0s 5ms/step - loss: 2.3022 - accuracy: 0.1080
Epoch 6/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3022 - accuracy: 0.1080
Epoch 7/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3021 - accuracy: 0.1080
Epoch 8/10
55/55 [==============================] - 0s 4ms/step - loss: 2.3021 - accuracy: 0.1080
Epoch 9/10
55/55 [==============================] - 0s 5ms/step - loss: 2.3021 - accuracy: 0.1080
Epoch 10/10
55/55 [==============================] - 0s 5ms/step - loss: 2.3020 - accuracy: 0.1080


In [ ]:
accuracy

0.09600000083446503

In [ ]:
_,accuracy_test = model.evaluate(x_test,y_testing, verbose = 0)

In [ ]:
accuracy_test

0.10000000149011612

In [ ]:
y_pred_test = model(x_test)
loss_object(y_pred_test,y_testing).numpy()##loss on test data

14.505448107389784

In [ ]:
loss_object(y_pred,y_val_array).numpy()##loss on validation

14.505051901994866

In [ ]:
best_param

[10, 1, 'relu', 1e-10]